In [1]:
import pandas as pd

In [2]:
# 1-1. 이동시간이 평균적으로 가장 오랠 걸린 구급대의 이동시간을 구하시오.
# - 분 단위로 환산 및 반올림하여 정수로 출력

df = pd.read_csv("https://raw.githubusercontent.com/YoungjinBD/dataset/main/data_6_1_1.csv")

df['출동시간'] = df['출동시간'].astype('datetime64[ns]')
df['도착시간'] = df['도착시간'].astype('datetime64[ns]')
df['이동시간'] = df['도착시간'] - df['출동시간']

print(round(max(df.groupby('구급대')['이동시간'].mean()).seconds/60))

83


In [3]:
# 1-2. 교수 1인당 맡고 있는 학생 수가 가장 많은 대학교를 찾고, 그 대학교의 전체 교수의 수를 구하시오. (정수로 출력)
df = pd.read_csv("https://raw.githubusercontent.com/YoungjinBD/dataset/main/data_6_1_2.csv")

In [4]:
df['학생수'] = df['경영학과'] + df['경제학과'] + df['컴퓨터공학과'] + df['산업공학과'] + df['국문학과'] + df['영문학과']
df['교수당학생수'] = df['학생수'] / df['교수']
df.sort_values('교수당학생수', ascending=False).iloc[0]

대학교명          대학교46
교수               22
경영학과            163
경제학과            189
컴퓨터공학과          192
산업공학과           121
국문학과             99
영문학과            183
학생수             947
교수당학생수    43.045455
Name: 45, dtype: object

In [5]:
# 1-3. 연도별 총 범죄 건수의 월평균 값을 구하고, 그 중 가장 큰 값을 출력하시오. (반올림하여 정수로 출력)

df = pd.read_csv("https://raw.githubusercontent.com/YoungjinBD/dataset/main/data_6_1_3.csv")

In [6]:
df['총범죄건수'] = df['폭력범죄'] + df['지능범죄'] + df['교통범죄'] + df['절도범죄'] + df['경제범죄'] + df['정치범죄']
df['연도'] = df['날짜'].apply(lambda x: x.split()[0])
df.groupby('연도')['총범죄건수'].mean().sort_values(ascending=False)

연도
2021년    528.333333
2018년    527.333333
2022년    524.416667
2019년    520.500000
2020년    514.916667
Name: 총범죄건수, dtype: float64

In [7]:
# 2. 학생구분 예측 모형을 만들어라.

train = pd.read_csv("https://raw.githubusercontent.com/YoungjinBD/dataset/main/data_6_2_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/YoungjinBD/dataset/main/data_6_2_test.csv")

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 530 entries, 0 to 529
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   일평균수면시간      530 non-null    float64
 1   일평균학습시간      530 non-null    float64
 2   일평균스마트폰사용시간  530 non-null    float64
 3   키            530 non-null    object 
 4   몸무게          530 non-null    float64
 5   체격           530 non-null    object 
 6   기초대사량        530 non-null    float64
 7   거주지역         530 non-null    object 
 8   구분           530 non-null    object 
dtypes: float64(5), object(4)
memory usage: 37.4+ KB


In [9]:
print(train['키'].unique())
print(train['체격'].unique())
print(train['거주지역'].unique())
print(train['구분'].unique())

['Tall' 'Short']
['L' 'M' 'XS' 'S']
['동구' '서구' '남구' '북구']
['고등학생' '중학생' '초등학생' '대학생' '유치원생']


In [10]:
train_dummies = pd.get_dummies(train[['키', '체격', '거주지역']])
train_combined = pd.concat([train, train_dummies], axis=1)
train_dropped = train_combined.drop(columns=['키', '체격', '거주지역'])

y_train = train_dropped['구분'].replace({'대학생': 0, '고등학생': 1, '중학생': 2, '초등학생': 3, '유치원생': 4})
X_train = train_dropped.drop(columns=['구분'])

/var/folders/kv/3bqrx9gs35v3nfj5z19_3bw00000gn/T/ipykernel_40647/2091784337.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_train = train_dropped['구분'].replace({'대학생': 0, '고등학생': 1, '중학생': 2, '초등학생': 3, '유치원생': 4})


In [11]:
X_train.shape, y_train.shape

((530, 15), (530,))

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

X_train_sub, X_validation, y_train_sub, y_validation = train_test_split(X_train, y_train, test_size=0.2)

rfc = RandomForestClassifier()
rfc.fit(X_train_sub, y_train_sub)

pred = rfc.predict(X_validation)
score = f1_score(y_validation, pred, average='macro')
print(score)

0.9558229813664596


In [13]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   일평균수면시간      230 non-null    float64
 1   일평균학습시간      230 non-null    float64
 2   일평균스마트폰사용시간  230 non-null    float64
 3   키            230 non-null    object 
 4   몸무게          230 non-null    float64
 5   체격           230 non-null    object 
 6   기초대사량        230 non-null    float64
 7   거주지역         230 non-null    object 
dtypes: float64(5), object(3)
memory usage: 14.5+ KB


In [14]:
print(test['키'].unique())
print(test['체격'].unique())
print(test['거주지역'].unique())

['Tall' 'Short']
['L' 'XS' 'S' 'M']
['북구' '남구' '동구' '서구']


In [15]:
test_dummies = pd.get_dummies(test[['키', '체격', '거주지역']])
test_combined = pd.concat([test, test_dummies], axis=1)
X_test = test_combined.drop(columns=['키', '체격', '거주지역'])

In [16]:
X_test.shape

(230, 15)

In [17]:
pred = rfc.predict(X_test)

converted_pred = []
convert = {
    0: '대학생',
    1: '고등학생',
    2: '중학생',
    3: '초등학생',
    4: '유치원생'
}

for p in pred:
    converted_pred.append(convert[p])

In [18]:
submit = pd.DataFrame({'구분예측': converted_pred})

In [19]:
# submit.to_csv("submit", index=False)ß

In [20]:
# 3-1-1. 항암제 투여 환자들 중 '무증상' 비율을 0과 1 사이로 구하시오.
# 3-1-2. 감기약의 부작용 비율과 항암제의 부작용 관찰 값이 통계적으로 유의미하게 차이가 있는지 카이제곱 검정을 이용하여 검정 통계량을 구하시오.
# 감기의 경우 아픔 5%, 조금아픔 10%, 속쓰림 15%, 무증상 70%
# 3-1-3. 위의 p-value를 구하시오.

df = pd.read_csv("https://raw.githubusercontent.com/YoungjinBD/dataset/main/data_6_3_1.csv")

In [21]:
df.value_counts()

항암제 
무증상     11
조금아픔     4
아픔       3
속쓰림      2
Name: count, dtype: int64

In [22]:
print('전체:', len(df))
print('유증상:',len(df[df['항암제']!='무증상']))
print('무증상:', len(df[df['항암제']=='무증상']))
print('무증상 비율:', len(df[df['항암제']=='무증상'])/len(df))

전체: 20
유증상: 9
무증상: 11
무증상 비율: 0.55


In [23]:
# 감기의 경우 아픔 5%, 조금아픔 10%, 속쓰림 15%, 무증상 70%
# obs: 아픔 3, 조금아픔 4, 속쓰림 2, 무증상 11
# exp: 아픔 1, 조금아픔 2, 속쓰림 3, 무증상 14

import scipy.stats as ss
ss.chisquare([3, 4, 2, 11], [1, 2, 3, 14])


Power_divergenceResult(statistic=6.976190476190476, pvalue=0.07266054733847577)

In [24]:
# 3-2-1. 선형회귀 모델을 만들어 독립변수 Ozone의 회귀계수를 구하시오.
# - 독립변수: Solar, Wind, Ozone
# - 종속변수: Temp
# 3-2-2. Solar와 Ozone은 고정, Wind가 증가함에 따라 Temp가 감소하는지 검증하기 위한 다중선형회귀분석을 수행하여 Wind의 회귀계수에 대한 p-value를 구하시오.
# 3-2-3. Solar가 150, Wind가 7.5, Ozone이 45일 경우, 예측 값과 그에 대한 95% 신뢰구간을 구하시오.

df = pd.read_csv("https://raw.githubusercontent.com/YoungjinBD/dataset/main/data_6_3_2.csv")

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Ozone   116 non-null    float64
 1   Solar   146 non-null    float64
 2   Wind    153 non-null    float64
 3   Temp    153 non-null    int64  
 4   Month   153 non-null    int64  
 5   Day     153 non-null    int64  
dtypes: float64(3), int64(3)
memory usage: 7.3 KB


In [26]:
df.dropna(inplace=True)

In [27]:
import statsmodels.api as sm

X = df[['Solar', 'Wind', 'Ozone']]
X = sm.add_constant(X)
y = df[['Temp']]

model = sm.OLS(y, X)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   Temp   R-squared:                       0.500
Model:                            OLS   Adj. R-squared:                  0.486
Method:                 Least Squares   F-statistic:                     35.65
Date:                Thu, 28 Nov 2024   Prob (F-statistic):           4.73e-16
Time:                        10:29:36   Log-Likelihood:                -368.79
No. Observations:                 111   AIC:                             745.6
Df Residuals:                     107   BIC:                             756.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         72.4186      3.216     22.522      0.000      66.044      78.793
Solar          0.0073      0.008      0.948      0.345      -0.008       0.022
Wind          -0.3229      0.233     -1.384      0.169      -0.785       0.139
Ozone          0.1720      0.026      6.516      0.000       0.120       0.224
==============================================================================
Omnibus:                        6.409   Durbin-Watson:                   1.170
Prob(Omnibus):                  0.041   Jarque-Bera (JB):                6.615
Skew:                          -0.591   Prob(JB):                       0.0366
Kurtosis:                       2.817   Cond. No.                     1.05e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.05e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [28]:
result.params['Ozone']

0.1719660419798164

In [29]:
result.pvalues['Wind']

0.16909868598497496

In [30]:
prediction = result.get_prediction([1, 150, 7.5, 45])
prediction.summary_frame()

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,78.826312,0.864464,77.112614,80.540011,65.171661,92.480964
